In [2]:
import tkinter as tk
import pandas as pd
import requests

In [3]:
def get_wikipedia_link(location_name):
    base_url = "http://en.wikipedia.org/wiki/"
    formatted_location = location_name.replace(" ", "_")
    return f"{base_url}{formatted_location}"

In [4]:
def generate_results(keywords, outputs=['title', 'url', 'body'], time=[], sent=[], countries=[], keyword_loc='body'):
    r = requests.get("http://eventregistry.org/api/v1/article/getArticles", {
        "action": "getArticles",
        "keyword": keywords,
        "articlesPage": 1,
        "articlesCount": 100,
        "articlesSortBy": "date",
        "articlesSortByAsc": False,
        "articlesArticleBodyLen": -1,
        "resultType": "articles",
        "dataType": [
            "news",
            "pr"
        ],
        "apiKey": "562eab60-8961-4906-82d9-6a5cbd74589c",
        "forceMaxDataTimeWindow": 31,
        "dateStart": time[0] if len(time)>0 else None,
        "dateEnd": time[1] if len(time)>0 else None,
        "sourceLocationUri": [get_wikipedia_link(country) for country in countries] if len(countries)>0 else None,
        "minSentiment" : sent[0] if len(sent)>0 else None,
        "maxSentiment" : sent[1] if len(sent)>0 else None,
        "keywordLoc" : keyword_loc
    }, headers={"content-type": "application/json"})
    
    articles = r.json()["articles"]["results"]

    # Créer une liste de dictionnaires pour construire la dataframe
    data = []
    for article in articles:
        data.append({x: article[x] for x in outputs})

    # Créer une dataframe pandas
    df = pd.DataFrame(data)

    return df

In [5]:
def save_to_csv(df, output_file_path):
    # Sauvegarder la dataframe dans un fichier CSV
    df.to_csv(output_file_path, index=False)

In [6]:
def on_results_button_click(entries):
    parameters = {'keywords': [], 'outputs': [], 'time': [], 'sent': [], 'countries': [], 'keyword_loc': ''}
    
    for entry, key in zip(entries, parameters.keys()):
        if entry.get():
            if key == 'sent':
                parameters[key] = [float(value) for value in entry.get().split()]
            elif key == 'keywords' or key == 'outputs' or key == 'countries':
                parameters[key] = entry.get().split(',')
            else:
                parameters[key] = entry.get()
    
    # Appeler generate_results pour obtenir la dataframe
    df = generate_results(**parameters)
    
    # Sauvegarder la dataframe dans un fichier CSV
    output_file_path = "resultats.csv"
    save_to_csv(df, output_file_path)

In [7]:
# Interface utilisateur améliorée
def create_label_entry(parent, label_text):
    frame = tk.Frame(parent)
    tk.Label(frame, text=label_text, width=20, anchor='w').pack(side='left')
    entry = tk.Entry(frame, width=30)
    entry.pack(side='left')
    frame.pack(pady=5)
    return entry

def interface_utilisateur():
    app.title("Collecte d'Informations")

    keywords_entry = create_label_entry(app, "Mots-clés:")
    outputs_entry = create_label_entry(app, "Colonnes de sortie:")
    time_entry = create_label_entry(app, "Période (YYYY-MM-DD YYYY-MM-DD):")
    sent_entry = create_label_entry(app, "Sentiment (x y):")
    countries_entry = create_label_entry(app, "Pays:")
    keyword_loc_entry = create_label_entry(app, "Emplacement des mots-clés:")

    entries = (keywords_entry, outputs_entry, time_entry, sent_entry, countries_entry, keyword_loc_entry)

    results_button = tk.Button(app, text="Résultats", command=lambda: on_results_button_click(entries))
    results_button.pack(pady=10)

interface_utilisateur()
app.mainloop()

TclError: can't invoke "wm" command: application has been destroyed

In [ ]:
def affichage_text():
    results_button = tk.Button(app, text="Résultats", command=lambda: on_results_button_click(*entries))
    results_button.pack()

In [ ]:
entries = interface_utilisateur()
affichage_text()
app.mainloop()